In [2]:
class Extension:
    def __init__(self, name, argument_type, arguments, function):
        self.name = name
        self.argument_type = argument_type #free
        self.arguments = arguments #dict: {x: Integer, y: Integer}
        self.function = function

In [4]:
def add(self, x, y):
    return(x+y)

In [6]:
addition = Extension("add", "free", {"x": "Integer", "y": "Integer"}, add)

In [37]:
initializer = " ".join(["""(assert({} require {}))""".format(addition.name, i) for i in list(addition.arguments.keys())])

In [39]:
print(initializer)

(assert(add require x)) (assert(add require y))


In [40]:
entry = """(defrule {name}
                        ?o <- (input ui {name})
                        =>
                        (retract ?o)
                        {initializer}
                    )""".format(name=addition.name, initializer=initializer)

In [41]:
print(entry)

(defrule add
                        ?o <- (input ui add)
                        =>
                        (retract ?o)
                        (assert(add require x)) (assert(add require y))
                    )


In [82]:
rules = []
for i in addition.arguments.keys():
    valid = "{'type' : 'regex', 'value' : '(\\\\d+)'}"
    require =  """(defrule {name}-{parameter_name}
                            ?a <- ({name} require {parameter_name})
                            => 
                            (retract ?a)
                            (assert (ui-state (text (str-cat "Please provide some value for {parameter_name}."))
                                                    (valid-answers "{valid}")
                                                    (clips-type "''")
                                                    (suggested-answers "")
                                                    (clipboard "''")
                                                    (fact-type (str-cat "{name} user-value {parameter_name}"))))
                            )""".format(name=addition.name, parameter_name=i, valid=valid)
    rules.append(require)

In [83]:
print(rules[0])

(defrule add-x
                            ?a <- (add require x)
                            => 
                            (retract ?a)
                            (assert (ui-state (text (str-cat "Please provide some value for x."))
                                                    (valid-answers "{'type' : 'regex', 'value' : '(\\d+)'}")
                                                    (clips-type "''")
                                                    (suggested-answers "")
                                                    (clipboard "''")
                                                    (fact-type (str-cat "add user-value x"))))
                            )


In [69]:
arguments = " ".join(["""?{}f <- ({} user-value {} ?{})""".format(i, addition.name, i, i) for i in list(addition.arguments.keys())])

In [70]:
print(arguments)

?xf <- (add user-value x ?x) ?yf <- (add user-value y ?y)


In [64]:
single_arguments = " ".join(["?{}".format(i) for i in list(addition.arguments.keys())])

In [65]:
print(single_arguments)

?x ?y


In [66]:
bind = """(bind ?r ({name} {single_arguments}))""".format(name=addition.name, single_arguments=single_arguments)

In [67]:
print(bind)

(bind ?r (add ?x ?y))


In [78]:
retract = " ".join(["(retract ?{}f)".format(i) for i in list(addition.arguments.keys())])

In [79]:
print(retract)

(retract ?xf) (retract ?yf)


In [80]:
complete = """(defrule {name}-complete
                    {arguments}
                    =>
                    {retract}
                    {bind}
                    (assert (ui-state (text (str-cat "The answer is " ?r))
                                    (valid-answers "''")
                                    (clips-type "''")
                                    (suggested-answers "")
                                    (clipboard "''")
                                    (fact-type "input ui"))))""".format(name=addition.name, arguments=arguments, parameter_name=i, valid=valid, bind=bind, retract=retract)

In [81]:
print(complete)

(defrule add-complete
                    ?xf <- (add user-value x ?x) ?yf <- (add user-value y ?y)
                    =>
                    (retract ?xf) (retract ?yf)
                    (bind ?r (add ?x ?y))
                    (assert (ui-state (text (str-cat "The answer is " ?r))
                                    (valid-answers "''")
                                    (clips-type "''")
                                    (suggested-answers "")
                                    (clipboard "''")
                                    (fact-type "input ui"))))
